In [52]:
import yfinance as yf
import pandas as pd
from datetime import timedelta

import plotly.graph_objects as go
apple = yf.download('AAPL', start='2018-01-01', end='2023-01-01')
df = apple

df['Date'] = df.index
#df=df.set_index(range(len(df)))
df.index = range(len(df))
df


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Date
0,42.540001,43.075001,42.314999,43.064999,40.722874,102223600,2018-01-02
1,43.132500,43.637501,42.990002,43.057499,40.715790,118071600,2018-01-03
2,43.134998,43.367500,43.020000,43.257500,40.904907,89738400,2018-01-04
3,43.360001,43.842499,43.262501,43.750000,41.370621,94640000,2018-01-05
4,43.587502,43.902500,43.482498,43.587502,41.216953,82271200,2018-01-08
...,...,...,...,...,...,...,...
1254,130.919998,132.419998,129.639999,131.860001,131.127060,63814900,2022-12-23
1255,131.380005,131.410004,128.720001,130.029999,129.307251,69007800,2022-12-27
1256,129.669998,131.029999,125.870003,126.040001,125.339409,85438400,2022-12-28
1257,127.989998,130.479996,127.730003,129.610001,128.889587,75703700,2022-12-29


In [53]:
import warnings
warnings.filterwarnings("ignore")

In [54]:
fig = go.Figure(data=[go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show()

In [55]:
def ifhammer(candle):
  if candle['High'] - candle['Open'] == 0:
        return 0
  ratio=(candle['Open']-candle['Low'])/(candle['High']-candle['Open'])
  if(ratio>2):
    return 1
  else: return 0

In [56]:
def ifhangingman(candle):
  if candle['Low'] - candle['Close'] == 0:
        return 0
  ratio=(candle['High']-candle['Open'])/(candle['Open']-candle['Low'])
  if(ratio>2):
    return 1
  else: return 0

In [57]:
def implement_strategy(data, initial_capital=1000, stop_loss_percentage=2):
    capital = initial_capital
    in_trade = False
    entry_index = 0
    peak_value = initial_capital
    max_drawdown = 0
    trade_log = pd.DataFrame(columns=['EntryIndex', 'ExitIndex', 'Duration', 'Returns', 'MaxDrawdown'])

    for i in range(1, len(data)):
        if ifhammer(data.iloc[i]) and not in_trade:  # Buy...Hammer
            in_trade = True
            entry_index = i
            stop_loss_price = data['Open'][entry_index] * (1 - stop_loss_percentage / 100)
            continue

        if (ifhangingman(data.iloc[i]) or data['Low'][i] < stop_loss_price) and in_trade:  # Sell...Hanging Man or Stop Loss
            in_trade = False
            exit_index = i
            duration = exit_index - entry_index
            returns = (data['Close'][exit_index - 1] / data['Open'][entry_index]) - 1

            peak_value = max(peak_value, capital * (1 + returns))
            drawdown = (capital - peak_value) / peak_value * 100   #to kind of make it independent of capital
            max_drawdown = min(max_drawdown, drawdown)

            trade_log = trade_log.append({
                'EntryIndex': entry_index,
                'ExitIndex': exit_index,
                'Duration': duration,
                'Returns': returns,
                'MaxDrawdown': max_drawdown
            }, ignore_index=True)

            capital *= (1 + returns)

    return trade_log, max_drawdown


In [58]:
trade_log, max_drawdown_strategy = implement_strategy(df)
max_drawdown_strategy = trade_log['MaxDrawdown'].min()
print(trade_log)

     EntryIndex  ExitIndex  Duration   Returns  MaxDrawdown
0           5.0        6.0       1.0 -0.001260     0.000000
1          15.0       16.0       1.0 -0.017094    -0.126039
2          22.0       23.0       1.0 -0.033133    -1.833334
3          25.0       26.0       1.0 -0.021767    -5.085844
4          27.0       28.0       1.0 -0.004202    -7.151854
..          ...        ...       ...       ...          ...
272      1246.0     1247.0       1.0 -0.026957   -96.833485
273      1248.0     1249.0       1.0 -0.032670   -96.918843
274      1250.0     1251.0       1.0 -0.020280   -97.019503
275      1253.0     1254.0       1.0 -0.015780   -97.079947
276      1255.0     1256.0       1.0 -0.010276   -97.126025

[277 rows x 5 columns]


In [59]:
print("\nMaximum Drawdown for the Entire Strategy:", max_drawdown_strategy)


Maximum Drawdown for the Entire Strategy: -97.12602493189105


In [60]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close']
)])

# Buy signals
fig.add_trace(go.Scatter(
    x=trade_log['EntryIndex'],
    y=df['Low'][trade_log['EntryIndex']],
    mode='markers',
    marker=dict(color='yellow', symbol='triangle-up', size=4),
    name='Buy Signal'
))

# Sell signals
fig.add_trace(go.Scatter(
    x=trade_log['ExitIndex'],
    y=df['High'][trade_log['ExitIndex']],
    mode='markers',
    marker=dict(color='blue', symbol='triangle-down', size=4),
    name='Sell Signal'
))

fig.show()

